<a href="https://colab.research.google.com/github/pvfalcao/A-semi-supervised-learning-approach-with-data-drift-detection-to-deal-with-ransomware/blob/main/1_Hidden_Layer_10S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação de bibliotecas


In [ ]:
import time
import psutil
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from torch.utils.data import Subset



Recebendo conjunto de dados


In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/Navarra/N10S10/N10S10.csv', header = None)
zero = pd.read_csv(r'/content/drive/MyDrive/Navarra/N10S10/zeroDays.csv', header = None)
user = pd.read_csv(r'/content/drive/MyDrive/Navarra/N10S10/userSamples_train.csv', header = None)

targets = df.iloc[:,df.shape[1] - 1]
df = df.iloc[:,0:-1]
tz = zero.iloc[:,zero.shape[1] - 1]
zero = zero.iloc[:,0:-1]
tu = user.iloc[:,user.shape[1] - 1]
user = user.iloc[:,0:-1]


Classes do Pytorch


In [ ]:
#prepare dataset for pytorch

class CSVDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        # Assume the last column is the target and the rest are features
        features = torch.tensor(row[:-1].values, dtype=torch.float32)
        features = features
        target = torch.tensor(row[-1], dtype=torch.long)
        target = target
        return features, target

# A custom Dataset class for in-memory data that takes features (DataFrame) and targets (Series)

class CustomTensorDataset(Dataset):
    def __init__(self, features_df, targets_series):
        # Convert pandas DataFrames/Series to torch Tensors and move to device
        self.features = torch.tensor(features_df.values, dtype=torch.float32)
        self.targets = torch.tensor(targets_series.values, dtype=torch.long)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

#The actual model

class NN1(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=2, dropout_rate=0.5):
        super(NN1, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        #self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        #x = self.softmax(x)
        return x

In [ ]:

#Initialize the variables

precision_list = []
recall_list = []
f1_list = []
accuracy_list = []

# Initialize 10-Fold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

#Consumption metrics

psutil.cpu_percent(interval=None)
start = time.process_time()

# Split the dataset into 10 folds

for fold, (train_idx, test_idx) in enumerate(skf.split(df, targets)):
    # Select data using .iloc to get pandas DataFrames/Series
    train_features = df.iloc[train_idx]
    train_labels = targets.iloc[train_idx]
    test_features = df.iloc[test_idx]
    test_labels = targets.iloc[test_idx]

    # Create instances of the new CustomTensorDataset
    train_dataset = CustomTensorDataset(train_features, train_labels)
    test_dataset = CustomTensorDataset(test_features, test_labels)

    # Create DataLoaders for this fold
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

    #Setup for 10S
    input_dim = 30
    output_dim = 2
    model = NN1(input_dim, output_dim)
    total_loss = 0
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 600
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for epoch in range(num_epochs):
        for batch_idx, (X, y) in enumerate(train_dataloader):
            optimizer.zero_grad()
            predicted = model(X)
            loss = criterion(predicted, y)
            loss.backward()
            optimizer.step()
            total_loss =+ loss.item()
            print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader)}")

    model.eval()
    with torch.no_grad():
        for X, y in test_dataloader:
            #predicted = model(X)
            #predicted = torch.argmax(predicted, dim = 1)
            predicted_logits = model(X)
            probabilities = torch.softmax(predicted_logits, dim=1) # Manually apply softmax for interpretation
            predicted_class = torch.argmax(probabilities, dim=1)
            #print(f"Sample Input: {sample_input}")

            # Calculating precision, recall, and F1 score using PyTorch
            TP = TP + ((predicted_class == 1) & (y == 1)).sum().item()
            FP = FP + ((predicted_class == 1) & (y == 0)).sum().item()
            FN = FN + ((predicted_class == 0) & (y == 1)).sum().item()
            TN = TN + ((predicted_class == 0) & (y == 0)).sum().item()


    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TN + TP)/(TP + FP + FN + TN)

    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    accuracy_list.append(accuracy)

#Consumption measures
end = time.process_time()
print('The processing time was:', end - start)
cpu_usage_system = psutil.cpu_percent(interval=None)
print(f"System-wide CPU usage: {cpu_usage_system}%")
print(f"Load average:{psutil.getloadavg()}")
print(f"CPU Frequency:{psutil.cpu_freq()}")
print("CPU usage (%):", psutil.cpu_percent())
print("CPU usage (%):", psutil.cpu_percent())
ram = psutil.virtual_memory()
print("RAM usage (%):", ram.percent)
print("RAM used (GB):", round(ram.used / 1e9, 2))

#Metrics
print(precision_list)
print(recall_list)
print(f1_list)
print(accuracy_list)
mean_precision = sum(precision_list) / len(precision_list)
mean_recall = sum(recall_list) / len(recall_list)
mean_f1 = sum(f1_list) / len(f1_list)
mean_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Precision: {precision:.5f}')
print(f'Recall: {recall:.5f}')
print(f'F1 Score: {f1:.5f}')
print(f'Accuracy: {accuracy:.5f}')
print(f'Mean precision: {mean_precision:.5f}')
print(f'Mean recall: {mean_recall:.5f}')
print(f'Mean F1 Score: {mean_f1:.5f}')
print(f'Mean accuracy: {mean_accuracy:.5f}')





A saída de streaming foi truncada nas últimas 5000 linhas.
Epoch 591, Loss: 0.0010513703462638924
Epoch 591, Loss: 0.0007526496720444309
Epoch 591, Loss: 0.0007128947853824914
Epoch 591, Loss: 0.0006570401304623686
Epoch 591, Loss: 0.0007494363411311027
Epoch 591, Loss: 0.0006785606643106985
Epoch 591, Loss: 0.0006035839382199425
Epoch 591, Loss: 0.0008472563573354363
Epoch 591, Loss: 0.0008040879376381907
Epoch 591, Loss: 0.0006163552919155044
Epoch 591, Loss: 0.0006705710905280052
Epoch 591, Loss: 0.0009773000124808
Epoch 591, Loss: 0.0006814089846307028
Epoch 591, Loss: 0.000866266276232749
Epoch 591, Loss: 0.0008351967308690639
Epoch 591, Loss: 0.0006538589361152579
Epoch 591, Loss: 0.0006619178315112283
Epoch 591, Loss: 0.0010119990355764365
Epoch 591, Loss: 0.00077215119354929
Epoch 591, Loss: 0.0005698006444072898
Epoch 591, Loss: 0.0006375700398221042
Epoch 591, Loss: 0.0008285303476295401
Epoch 591, Loss: 0.0009149365086373085
Epoch 591, Loss: 0.0007840717425111864
Epoch 591, 

Teste Zero day

In [ ]:
test_zeroday = CustomTensorDataset(zero, tz)
zeroday_dataloader = DataLoader(test_zeroday, batch_size=64, shuffle=True)
TP = 0
FP = 0
FN = 0
TN = 0
for X, y in zeroday_dataloader:
   #predicted = model(X)
            #predicted = torch.argmax(predicted, dim = 1)
            predicted_logits = model(X)
            probabilities = torch.softmax(predicted_logits, dim=1) # Manually apply softmax for interpretation
            predicted_class = torch.argmax(probabilities, dim=1)
            #print(f"Sample Input: {sample_input}")

            # Calculating precision, recall, and F1 score using PyTorch
            TP = TP + ((predicted_class == 1) & (y == 1)).sum().item()
            FP = FP + ((predicted_class == 1) & (y == 0)).sum().item()
            FN = FN + ((predicted_class == 0) & (y == 1)).sum().item()
            TN = TN + ((predicted_class == 0) & (y == 0)).sum().item()


precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (TN + TP)/(TP + FP + FN + TN)

print(f'Precision: {precision:.5f}')
print(f'Recall: {recall:.5f}')
print(f'F1: {f1:.5f}')
print(f'Accuracy: {accuracy:.5f}')

Precision: 1.00000
Recall: 1.00000
F1: 1.00000
Accuracy: 1.00000


Teste amostras benignas

In [ ]:
test_user = CustomTensorDataset(user, tu)
user_dataloader = DataLoader(test_user, batch_size=64, shuffle=True)
TP = 0
FP = 0
FN = 0
TN = 0
for X, y in user_dataloader:
    #predicted = model(X)
            #predicted = torch.argmax(predicted, dim = 1)
            predicted_logits = model(X)
            probabilities = torch.softmax(predicted_logits, dim=1) # Manually apply softmax for interpretation
            predicted_class = torch.argmax(probabilities, dim=1)
            #print(f"Sample Input: {sample_input}")

            # Calculating precision, recall, and F1 score using PyTorch
            TP = TP + ((predicted_class == 1) & (y == 1)).sum().item()
            FP = FP + ((predicted_class == 1) & (y == 0)).sum().item()
            FN = FN + ((predicted_class == 0) & (y == 1)).sum().item()
            TN = TN + ((predicted_class == 0) & (y == 0)).sum().item()


precision = TN / (TN + FN) if TN + FN > 0 else 0
recall = TN / (TN + FP) if TN + FP > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (TN + TP)/(TP + FP + FN + TN)

print(f'Precision: {precision:.5f}')
print(f'Recall: {recall:.5f}')
print(f'F1: {f1:.5f}')
print(f'Accuracy: {accuracy:.5f}')

Precision: 1.00000
Recall: 0.10488
F1: 0.18985
Accuracy: 0.10488
